In [4]:
import pandas as pd
import numpy as np
import artm
from tqdm import tqdm_notebook
import time

In [21]:
with open('logs.txt') as f_in:
    logs = f_in.readlines()

In [22]:
small_logs = logs[::2]

In [32]:
small_logs = small_logs[::2]

In [33]:
len(small_logs)

621712

In [34]:
with open('small_log.txt', 'w') as f_out:
    for i in (small_logs):
        f_out.write(i)

In [35]:
bach_vectorizer = artm.BatchVectorizer(data_path='small_log.txt',
                                      data_format='vowpal_wabbit', target_folder='batch_small',
                                      batch_size=20)

In [36]:
T = 30   # количество тем
topic_names=["sbj"+str(i) for i in range(T-1)]+["bcg"]

model_artm = artm.ARTM(num_topics=T, topic_names=topic_names, reuse_theta=True,
                       num_document_passes=1)

In [38]:
np.random.seed(1)
dictionary = artm.Dictionary()
dictionary.gather(data_path=bach_vectorizer.data_path)
model_artm.initialize(dictionary)

In [39]:
model_artm.scores.add(artm.TopTokensScore(name='metric1', num_tokens=15))

In [40]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='smoothing', dictionary=dictionary,
                                                         topic_names='bcg', tau=1e5))

In [41]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=30)

In [42]:
model_artm.regularizers.add(artm.SmoothSparsePhiRegularizer(name='stimulates',
                                                            dictionary=dictionary,
                                                            topic_names=["sbj"+str(i) for i in range(0, 29)], tau=-1e5))

In [43]:
model_artm.fit_offline(batch_vectorizer=bach_vectorizer, num_collection_passes=15)

In [44]:
for topic_name in model_artm.topic_names:
    print (topic_name + ':')
    for word in model_artm.score_tracker["metric1"].last_tokens[topic_name]:
        print(word, end=' ')
    print()

sbj0:
отзывы инструкция яндекс инстаграм каталог телефон мтс com применению работа работы то рецепт город когда 
sbj1:
и фильм о я 3 при 2017 песни или инструкция во тв снять чем какой 
sbj2:
цена магазин интернет мир матч музыка нет samsung песня рождения мой ты замена шоу размер 
sbj3:
ютуб метро курс схема википедия елена почему видеохостинг работает доллара платье новосибирске лето лучшие быть 
sbj4:
купить 5 цена отзывы москва расписание инструкция цены дом лет применению 8 москвы новгород сочи 
sbj5:
сайт официальный личный кабинет это ru за 10 кто или вход www ржд 11 читать 
sbj6:
от 3 ру сезон майл 8 области про почта рф серия время сделать 12 ремонт 
sbj7:
2018 из россия июня хорватия июля сериал кто инструкция википедия года екатеринбург испания применению 14 
sbj8:
с порно видео до 6 телефон рецепт игра 18 инстаграм аниме майнкрафт торт 06 характеристики 
sbj9:
скачать секс магазин интернет песня игры музыку the район песню of стоимость девушка во d0 
sbj10:
в на для 2018 по